In [12]:
import wget

url = "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040"
filename = wget.download(url, 'tennis_court_det_dataset.zip')
print(f"Downloaded {filename}")

Downloaded tennis_court_det_dataset.zip


In [13]:
# unzip tennis_court_det_dataset.zip
import zipfile

# The actual path to your downloaded zip file
zip_path = r'C:\Users\cutle\Forward_Op\tennis_cv\training\tennis_court_det_dataset.zip'

# The directory where you want to extract the contents of the zip file
extract_to = r'C:\Users\cutle\Forward_Op\tennis_cv\training'

# Using the zipfile module to extract all the contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Extracted to {extract_to}")



Extracted to C:\Users\cutle\Forward_Op\tennis_cv\training


In [14]:
import os

# The directory you want to check
directory_path = r'C:\Users\cutle\Forward_Op\tennis_cv\training\data'

# List all files and directories in the specified path
contents = os.listdir(directory_path)

print("Contents of the directory:")
for item in contents:
    print(item)


Contents of the directory:
data_train.json
data_val.json
images


# Start Code

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

c:\Users\cutle\miniconda3\envs\The_Matrix\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [3]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((244, 244)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [4]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create Model

In [5]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

c:\Users\cutle\miniconda3\envs\The_Matrix\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\cutle\miniconda3\envs\The_Matrix\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model = model.to(device)

# Train Model

In [7]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [8]:
epochs=20
for epoch in range(epochs):
    for i, (img,kps) in enumerate(train_loader):
        imgs = img.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14991.44921875
Epoch 0, iter 10, loss: 15100.185546875
Epoch 0, iter 20, loss: 14721.232421875
Epoch 0, iter 30, loss: 13729.1630859375
Epoch 0, iter 40, loss: 13809.388671875
Epoch 0, iter 50, loss: 12695.853515625
Epoch 0, iter 60, loss: 13510.5791015625
Epoch 0, iter 70, loss: 12006.9267578125
Epoch 0, iter 80, loss: 12301.89453125
Epoch 0, iter 90, loss: 11275.017578125
Epoch 0, iter 100, loss: 10747.2177734375
Epoch 0, iter 110, loss: 11384.7666015625
Epoch 0, iter 120, loss: 10547.169921875
Epoch 0, iter 130, loss: 9668.1044921875
Epoch 0, iter 140, loss: 9387.84765625
Epoch 0, iter 150, loss: 8627.193359375
Epoch 0, iter 160, loss: 9228.984375
Epoch 0, iter 170, loss: 8714.4052734375
Epoch 0, iter 180, loss: 8058.29052734375
Epoch 0, iter 190, loss: 8030.7939453125
Epoch 0, iter 200, loss: 7703.416015625
Epoch 0, iter 210, loss: 7330.802734375
Epoch 0, iter 220, loss: 6812.03076171875
Epoch 0, iter 230, loss: 6787.81591796875
Epoch 0, iter 240, loss: 6561.

In [9]:
torch.save(model.state_dict(), "keypoints_model.pt")